### Model_Train_00

### 패키지 설치

In [ ]:
!pip install facenet-pytorch

In [ ]:
!pip install --upgrade torch torchvision

## 데이터 전처리

### 패키지

In [2]:
import os
import zipfile
import cv2
import torch
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm
from google.colab import drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 비디오 zip 파일 폴더 및 저장 폴더 경로
video_folder = '/content/drive/MyDrive/datasets/Video_maindata'  # 비디오가 저장될 폴더
output_folder = '/content/drive/MyDrive/datasets/Image_maindata'  # 얼굴 이미지 저장 폴더

# 압축 파일 리스트
zip_file_paths = [
    '/content/drive/MyDrive/datasets/Main_DATA/dfdc_train_part_00.zip',
    '/content/drive/MyDrive/datasets/Main_DATA/dfdc_train_part_01.zip',
    '/content/drive/MyDrive/datasets/Main_DATA/dfdc_train_part_02.zip',
    '/content/drive/MyDrive/datasets/train_sample_videos.zip'
]

# 저장 폴더가 없으면 생성
os.makedirs(video_folder, exist_ok=True)

# 각 ZIP 파일 압축 해제
for zip_file_path in zip_file_paths:
    print(f"Extracting {zip_file_path}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(video_folder)
    print(f"Finished extracting {zip_file_path}")

print("모든 ZIP 파일이 성공적으로 압축 해제되었습니다.")


In [4]:
# Step 4: GPU 또는 CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

사용 중인 장치: cuda


In [5]:
# Step 5: GPU를 사용한 MTCNN 초기화
mtcnn = MTCNN(keep_all=True, device=device)

/usr/local/lib/python3.10/dist-packages/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
/usr/local

In [6]:
# Step 6: 추가 설정
frame_interval = 10  # 프레임 간격
resize_dim = (224, 224)  # 얼굴 이미지 크기

In [7]:
# Step 7: 비디오 처리 함수
def process_video(video_path, output_folder, mtcnn, frame_interval, resize_dim):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_count = 0
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # 비디오별 하위 폴더 생성
    video_output_folder = os.path.join(output_folder, video_name)
    os.makedirs(video_output_folder, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 간격에 따라 샘플링
        if frame_count % frame_interval == 0:
            # OpenCV BGR 이미지를 PIL RGB로 변환
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)

            # MTCNN을 통한 얼굴 감지
            boxes, _ = mtcnn.detect(pil_img)
            if boxes is not None:
                for i, box in enumerate(boxes):
                    # 얼굴 크롭 및 리사이즈
                    face = pil_img.crop((box[0], box[1], box[2], box[3])).resize(resize_dim)
                    face.save(os.path.join(video_output_folder, f"{video_name}_face_{extracted_count}.jpg"))
                    extracted_count += 1

        frame_count += 1

    cap.release()

In [10]:
# Step 8: 하위 폴더의 모든 비디오 파일을 가져오기
video_files = []
for root, dirs, files in os.walk(video_folder):
    for file in files:
        if file.endswith('.mp4'):
            video_files.append(os.path.join(root, file))

if not video_files:
    print("No .mp4 files found in the extracted folders. Check if the ZIP files contain .mp4 videos.")
else:
    print(f"총 {len(video_files)}개의 비디오 파일이 발견되었습니다.")

총 15128개의 비디오 파일이 발견되었습니다.


In [ ]:
# Step 9: 모든 비디오 파일 처리
for folder in os.listdir(video_folder):
    folder_path = os.path.join(video_folder, folder)
    if os.path.isdir(folder_path):
        video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]
        for video_file in tqdm(video_files, desc=f"{folder} 비디오 처리 중"):
            video_path = os.path.join(folder_path, video_file)
            process_video(video_path, output_folder, mtcnn, frame_interval, resize_dim)
            print(f"{video_file} 처리 완료")

## Model Train

### 패키지 설치

In [ ]:
# EfficientNet-PyTorch 설치
!pip install efficientnet-pytorch

# pretrainedmodels 설치 (Xception 모델용)
!pip install pretrainedmodels


In [28]:
import os
import ssl
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.models import ResNet50_Weights
from efficientnet_pytorch import EfficientNet
from pretrainedmodels import xception
from PIL import Image
from tqdm import tqdm
from google.colab import drive
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [26]:
# SSL 인증서 검증 비활성화
ssl._create_default_https_context = ssl._create_unverified_context

# 커스텀 데이터셋 정의
class FaceForensicsDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = []
        self.labels = []

        # 하위 디렉터리까지 포함하여 이미지와 레이블을 로드
        for root, _, files in os.walk(image_folder):
            for img in files:
                if img.endswith(('.jpg', '.png')):
                    img_path = os.path.join(root, img)
                    self.images.append(img_path)
                    # 파일명에 "REAL" 포함 여부로 라벨을 설정
                    self.labels.append(1 if "REAL" in img else 0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # NaN 검사
        if torch.isnan(image).any() or torch.isinf(image).any():
            print(f"Data issue in image {img_path}")
        return image, label

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 커스텀 데이터셋 인스턴스 생성
image_folder = '/content/drive/MyDrive/datasets/Image_maindata'
dataset = FaceForensicsDataset(image_folder=image_folder, transform=transform)

# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [29]:
# GPU 장치 설정 (Colab에서는 'cuda' 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 설정
xception_model = xception(num_classes=1000, pretrained='imagenet')
xception_model.last_linear = nn.Linear(xception_model.last_linear.in_features, 2)
xception_model = xception_model.to(device)

efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
efficientnet_model = efficientnet_model.to(device)

resnet_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)
resnet_model = resnet_model.to(device)  # 모델을 MPS에 올리기
# 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = Adam(
    list(xception_model.parameters()) +
    list(efficientnet_model.parameters()) +
    list(resnet_model.parameters()),
    lr=1e-5  # 학습률을 낮추어 설정
)

# NaN 검사 함수
def check_nan(tensor, name=""):
    if torch.isnan(tensor).any():
        print(f"NaN detected in {name}")
        return True
    return False

# 앙상블 예측 함수
def ensemble_predict(models, images):
    outputs = [model(images) for model in models]
    outputs = torch.stack(outputs).mean(dim=0)
    return outputs

# 학습 및 검증 함수 정의
def train_ensemble_model(models, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        for model in models:
            model.train()

        train_loss, train_correct = 0.0, 0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = ensemble_predict(models, images)

            # NaN 검사
            if check_nan(outputs, "outputs"):
                return

            loss = criterion(outputs, labels)
            if check_nan(loss, "loss"):
                return

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        train_acc = train_correct / len(train_dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_acc:.4f}")

        # Validation
        for model in models:
            model.eval()

        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
                images, labels = images.to(device), labels.to(device)
                outputs = ensemble_predict(models, images)
                if check_nan(outputs, "validation outputs"):
                    return

                loss = criterion(outputs, labels)
                if check_nan(loss, "validation loss"):
                    return

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

        val_acc = val_correct / len(val_dataset)
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_acc:.4f}")




Using device: cuda
Loaded pretrained weights for efficientnet-b0


In [ ]:
# 학습 시작
models = [xception_model, efficientnet_model, resnet_model]
train_ensemble_model(models, criterion, optimizer, num_epochs=5)

Training Epoch 1/5:   0%|          | 16/9085 [03:23<32:38:24, 12.96s/it]

In [ ]:
# 모델 저장 경로 설정
output_dir = '/content/drive/MyDrive/datasets'
os.makedirs(output_dir, exist_ok=True)

# 모델 저장
torch.save(xception_model.state_dict(), os.path.join(output_dir, 'xception_model_02.pth'))
torch.save(efficientnet_model.state_dict(), os.path.join(output_dir, 'efficientnet_model_02.pth'))
torch.save(resnet_model.state_dict(), os.path.join(output_dir, 'resnet_model_02.pth'))

print("모델이 성공적으로 저장되었습니다.")